In [17]:
from maraboupy import Marabou, MarabouCore
import numpy as np
import pandas as pd
import csv
import os
import sys

In [18]:
def write_values_to_csv(values_dict, filename):
    # 获取当前脚本的目录
    current_directory = os.path.dirname(os.path.abspath(__file__))

    # 将文件名与当前目录连接起来
    full_path = os.path.join(current_directory, filename)

    # 打开文件进行追加
    with open(full_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # 提取字典中的值并写入一行
        values = list(values_dict.values())
        writer.writerow(values)

    print(f"Values written to {full_path}")


In [ ]:
def main():
    time = 0
    count = 0
    # data = pd.read_csv(r'C:\Users\shinm\OneDrive - University of Bristol\Courses\DissertationProject\Experiment\Code\Marabou\Data_STS_modified.csv')
    data = pd.read_csv(
        r'/home/adam/Experiment/Code/Marabou/Data_STS_modified.csv')
    # Names
    names = open(
        "/home/adam/Experiment/Datasets/STS_data/Processing/Data_Names.txt", "r")
    # names = open("C:\\Users\\shinm\\OneDrive - University of Bristol\\Courses\\DissertationProject\\Experiment\\Datasets\\STS_data\\Processing\\Data_Names.txt", "r")

    # names = open("Data_Names_Gender.txt", "r")
    contents = names.read()
    dictionary = ['Stand-to-stand time', 'Sit-to-stand time', 'Stand-to-sit time', 'M_hip vertical range', 'M_hip depth range', 'M_hip max vertical velocity',
                  'M_hip min vertical velocity', 'M_hip max depth velocity', 'M_hip min depth velocity', 'Knee flexo-extension range', 'Knee flexo-extension max velocity', 'Knee flexo-extension min velocity',
                  'Hip flexo-extension range', 'Hip flexo-extension max velocity', 'Hip flexo-extension min velocity', 'Hip abduction-adduction range', 'Hip abduction-adduction max velocity', 'Hip abduction-adduction min velocity',
                  'Ankle flexo-extension range', 'Ankle flexo-extension max velocity', 'Ankle flexo-extension min velocity', 'M_shoulder vertical range', 'M_shoulder depth range', 'M_shoulder max vertical velocity',
                  'M_shoulder min vertical velocity', 'M_shoulder max depth velocity', 'Spine flexo-extension range', 'Spine flexo-extension max velocity', 'Spine flexo-extension min velocity', 'Spine abduction-adduction range',
                  'Spine abduction-adduction max velocity', 'Spine abduction-adduction min velocity', 'Heart Rate']
    data.columns = ['Stand-to-stand time', 'Sit-to-stand time', 'Stand-to-sit time',
                    'M_hip vertical range', 'M_hip depth range', 'M_hip max vertical velocity',
                    'M_hip min vertical velocity', 'M_hip max depth velocity', 'M_hip min depth velocity',
                    'Knee flexo-extension range', 'Knee flexo-extension max velocity', 'Knee flexo-extension min velocity',
                    'Hip flexo-extension range', 'Hip flexo-extension max velocity', 'Hip flexo-extension min velocity',
                    'Hip abduction-adduction range', 'Hip abduction-adduction max velocity', 'Hip abduction-adduction min velocity',
                    'Ankle flexo-extension range', 'Ankle flexo-extension max velocity', 'Ankle flexo-extension min velocity',
                    'M_shoulder vertical range', 'M_shoulder depth range', 'M_shoulder max vertical velocity',
                    'M_shoulder min vertical velocity', 'M_shoulder max depth velocity', 'Spine flexo-extension range',
                    'Spine flexo-extension max velocity', 'Spine flexo-extension min velocity', 'Spine abduction-adduction range',
                    'Spine abduction-adduction max velocity', 'Spine abduction-adduction min velocity', 'Heart Rate',
                    'Fatigue Condition']

    data = pd.DataFrame(data)
    data = data.drop(columns=['M_hip depth range', 'M_hip min vertical velocity', 'M_hip max depth velocity', 'M_hip min depth velocity',
                              'Knee flexo-extension min velocity', 'Hip flexo-extension min velocity',
                              'Hip abduction-adduction min velocity', 'Ankle flexo-extension min velocity',
                              'M_shoulder depth range', 'M_shoulder min vertical velocity',
                              'M_shoulder max depth velocity', 'Spine flexo-extension min velocity',
                              'Spine abduction-adduction min velocity'])

    # bounds = pd.read_csv(
    #     '/Code/Marabou/Behaviour_analysis_misclassification/bounds_3std.csv')
    # bounds = pd.read_csv(
    #     '/home/adam/Experiment/Code/Marabou/Behaviour_analysis_misclassification/bounds_fuzz90.csv'
    # )

In [19]:
file_name = 'exo_model_without_softmax'
network = Marabou.read_tf(file_name, modelType='savedModel_v2')

In [20]:
# bounds = pd.read_csv(
#     '/home/adam/Experiment/Code/Marabou/Behaviour_analysis_misclassification/Bounds/HR-STS/bounds_mixed_inputspacetop90.csv'
#     )
# high_fatigue = pd.DataFrame(bounds)

file_name = 'model_without_softmax.onnx'
# network = Marabou.read_onnx(file_name, inputNames='dense_0_input', outputNames='dense_62', reindexOutputVars=True)
network = Marabou.read_onnx(file_name)

In [ ]:

# High fatigue query
inputVars = network.inputVars[0][0]

# Set the upper bound of each input variable to the high_fatigue upper bound
for i, (var_name, bounds) in enumerate(high_fatigue.iterrows()):
    network.setUpperBound(inputVars[i], bounds['upper_bound'])
    network.setLowerBound(inputVars[i], bounds['lower_bound'])

# Set the lower and upper bounds for the output variable(0-6 means low or medium fatigue)
outputVars = network.outputVars[0]
for i in range(len(outputVars)):
    for j in range(len(outputVars[i])):
        network.setLowerBound(outputVars[i][j], 0)
        network.setUpperBound(outputVars[i][j], 6)

options = Marabou.createOptions(numWorkers=20, initialTimeout=5, initialSplits=100, onlineSplits=100,
                                timeoutInSeconds=1800, timeoutFactor=1.5,
                                verbosity=2, snc=True, splittingStrategy='auto',
                                sncSplittingStrategy='auto', restoreTreeStates=False,
                                splitThreshold=20, solveWithMILP=False, dumpBounds=True)
# Execute query in a loop
try:

    while True:
        result = network.solve(
            filename='/home/adam/Experiment/Code/Marabou/logs/Behviour_analysis_mixedtop90.log',
            options=options)
        status, values, stats = result
        write_values_to_csv(values, 'HR-STSmixed_results/highfatigue_mixedtop90_005.csv')
        time += stats.getTotalTimeInMicro()
        count += 1

        # if time > 180000000:
        #     print("Timeout.")
        #     print("Time:", time)
        #     break

        # If return 'sat', which means there are some high fatigue data in the dataset is classified as low or medium fatigue
        if status == "sat":
            print("Solution: ", count, " found!")
            # print("Values:", values)

            print("Time:", time)

            # If result is SAT, add a new constraint to exclude the current solution
            for i in range(len(inputVars)):
                # Create a constraint that the variable is less than the current solution
                eq1 = MarabouCore.Equation(MarabouCore.Equation.LE)
                eq1.addAddend(1, inputVars[i])
                # Subtract a small value to avoid the current solution
                eq1.setScalar(values[inputVars[i].item()] - 0.05)

                # Create a constraint that the variable is greater than the current solution
                eq2 = MarabouCore.Equation(MarabouCore.Equation.GE)
                eq2.addAddend(1, inputVars[i])
                # Add a small value to avoid the current solution
                eq2.setScalar(values[inputVars[i].item()] + 0.05)

                # Add the disjunction of the two constraints to the network
                network.addDisjunctionConstraint([[eq1], [eq2]])

        elif status == "unsat":
            print("No solution found.")
            print("Time:", time)
            break

        pass

except KeyboardInterrupt:
    print("Interrupted")
    sys.exit()

